In [ ]:
# bayes opt
# 1d-cnn
# table net
# 完整版NN

In [3]:
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import GridSearchCV
import numpy as np

In [5]:
x, y = make_classification(n_samples=1000,n_features=10,n_classes=2)
# 先看看不调参的结果：
rf = RandomForestClassifier()
print(np.mean(cross_val_score(rf, x, y, cv=20, scoring='roc_auc')))


0.9824786538461538


array([[-0.63091039,  1.00160635,  1.03387296, ...,  0.0399342 ,
         0.11741414, -0.2179988 ],
       [-0.95714891,  1.82470542,  0.67989361, ..., -1.00322079,
         0.04179445,  0.34729875],
       [ 0.2167972 , -0.64931554, -1.88034366, ...,  1.04994561,
         0.03217544, -0.26051524],
       ...,
       [ 1.79702021, -3.31458362,  1.39541898, ...,  1.49571797,
        -0.84752387, -1.04577427],
       [ 1.30659346, -2.43112717, -0.06306665, ...,  1.16117693,
         0.59991044,  0.27413139],
       [ 1.84639708, -3.39715448, -0.11916176, ...,  1.50717136,
         0.38300915, -1.2043807 ]])

In [7]:
# bayes调参初探
def rf_cv(n_estimators, min_samples_split, max_features, max_depth):
    val = cross_val_score(
        RandomForestClassifier(n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_features=min(max_features, 0.999), # float
            max_depth=int(max_depth),
            random_state=2
        ),
        x, y, scoring='roc_auc', cv=5
    ).mean()
    return val
# 实例化一个bayes优化对象
rf_bo = BayesianOptimization(
        rf_cv,
        {'n_estimators': (10, 250),
        'min_samples_split': (2, 25),
        'max_features': (0.1, 0.999),
        'max_depth': (5, 15)}
    )
# 最优化
rf_bo.maximize()


|   iter    |  target   | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  1        |  0.9794   |  14.65    |  0.9376   |  25.0     |  159.2    |
|  2        |  0.9822   |  13.32    |  0.7112   |  22.28    |  117.5    |
|  3        |  0.9836   |  7.284    |  0.6625   |  7.045    |  156.3    |
|  4        |  0.9838   |  10.56    |  0.6075   |  9.206    |  200.9    |
|  5        |  0.9804   |  13.25    |  0.9453   |  21.49    |  74.22    |
|  6        |  0.9727   |  10.07    |  0.1462   |  9.556    |  201.9    |
|  7        |  0.9816   |  7.475    |  0.2469   |  7.472    |  156.2    |
|  8        |  0.9839   |  7.87     |  0.3609   |  7.32     |  156.0    |
|  9        |  0.9836   |  7.591    |  0.6007   |  7.168    |  156.8    |
|  10       |  0.9816   |  8.219    |  0.8609   |  7.254    |  155.8    |
|  11       |  0.9812   |  10.35    |  0.2633   |  8.669    |  201.3    |
|  12       |  0.9837   |  8.462    | 

In [9]:
print(rf_bo.max)
cross_val_score(
        RandomForestClassifier(n_estimators=249,
            min_samples_split=24,
            max_features=0.399, # float
            max_depth=13,
            random_state=2
        ),
        x, y, scoring='roc_auc', cv=5
    ).mean()

{'target': 0.984399703970397, 'params': {'max_depth': 6.80420069738127, 'max_features': 0.3778588084783545, 'min_samples_split': 6.671318825833385, 'n_estimators': 157.4017409926933}}


0.9824395859585959

In [10]:
import xgboost as xgb 
def xgb_cv(n_estimators, max_depth,learning_rate,min_child_weight,max_delta_step,subsample,colsample_bytree,reg_alpha,reg_lambda,scale_pos_weight):
    val = cross_val_score(
        xgb.XGBClassifier(max_depth=int(max_depth),
            learning_rate=min(learning_rate,0.3),
            n_estimators=int(n_estimators),
            silent=True,
            objective='binary:logitraw',
            #num_class=2 ,          
            nthread=-1,
            gamma=0,
            min_child_weight=int(min_child_weight),
            max_delta_step=int(max_delta_step),
            subsample=min(subsample,0.95),
            colsample_bytree=min(colsample_bytree,0.95),
            colsample_bylevel=1,
            reg_alpha=min(reg_alpha,1),
            reg_lambda=min(reg_lambda,1),
            scale_pos_weight=min(scale_pos_weight,1),
            seed=0,
            missing=None),
        x, y, scoring='roc_auc', cv=5
    ).mean()
    return val 

xgb_bo = BayesianOptimization(
        xgb_cv,
        {
              'max_depth': (5,100),
              'learning_rate': (0.001,0.3),
              'n_estimators': (500,5000),
              'min_child_weight': (0,20),
              'max_delta_step': (0,2),
              'subsample': (0.1,0.99),
              'colsample_bytree': (0.1,0.99),
              'reg_alpha': (0,1),
              'reg_lambda': (0,1),
              'scale_pos_weight': (0,1)

}
    )
xgb_bo.maximize(n_iter=95)
xgb_bo.max

|   iter    |  target   | colsam... | learni... | max_de... | max_depth | min_ch... | n_esti... | reg_alpha | reg_la... | scale_... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------
[11:40:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




c:\zhangli\software\installer\python38\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\zhangli\software\installer\python38\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:40:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




c:\zhangli\software\installer\python38\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:40:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




...

c:\zhangli\software\installer\python38\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:52:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




c:\zhangli\software\installer\python38\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:52:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


|  100      |  0.9784   |  0.8256   |  0.1764   |  0.02661  |  70.31    |  1.126    |  3.831e+0 |  0.9305   |  0.2335   |  0.3632   |  0.7662   |


{'target': 0.9832996139613961,
 'params': {'colsample_bytree': 0.99,
  'learning_rate': 0.3,
  'max_delta_step': 2.0,
  'max_depth': 5.0,
  'min_child_weight': 0.0,
  'n_estimators': 2609.4854553296364,
  'reg_alpha': 1.0,
  'reg_lambda': 1.0,
  'scale_pos_weight': 1.0,
  'subsample': 0.99}}